# Simple sudoku solver using logical constraints (with GUI)
[![sudoku.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/miscellaneous\sudoku.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/miscellaneous\sudoku.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/miscellaneous\sudoku.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/miscellaneous\sudoku.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/miscellaneous\sudoku.ipynb)

Description: Simple sudoku solver using the *alldiff* operator; needs a solver supporting constraints programming or a MIP solver with automatic reformulation support (see [here](https://amplmp.readthedocs.io/) for more information). A little GUI implemented using ipywidgets helps with data visualization and specification.

Tags: amplpy, constraint-programming, GUI

Notebook author: Christian Valente <<christian.valente@gmail.com>>

Model author: Christian Valente

In [ ]:
# Install dependencies (using ipywidgets for the simple GUI)
!pip install -q amplpy ipywidgets

In [ ]:
# Google Colab & Kaggle integration
MODULES=['ampl', 'highs'] # need a solver supporting the "alldiff" operator
from amplpy import tools
ampl = tools.ampl_notebook(modules=MODULES, globals_=globals()) # instantiate AMPL object and register magics

### Define the AMPL model of a sudoku game. 
In this example, we want to show how to use the logical operator "alldiff" to avoid the explicit use of binary variables. Note that the model is parametric: **BASE** defines the size of the (square) subgrids making up the game. For a normal sudoku game, where the numbers go from 1 to 9, this has to be set to 3.

In [ ]:
%%ampl_eval
param BASE;
param L := BASE*BASE;

set ROWS := {1..L};
set COLS := {1..L};

# The variables containing the numbers that make up the sudoku.
var n{ROWS, COLS} >=1, <=L integer;

# This indexed set memorizes the tuples of coordinates for each 
# sub-square making up the grid
set SUBSQUARES{sr in 1..BASE, sc in 1..BASE} within {ROWS, COLS}
	= {(sr-1)*BASE+1..sr*BASE, (sc-1)*BASE+1..sc*BASE};

# Set this parameter to non-zero to force that position to have
# that value
param givenData{ROWS, COLS} default 0;

# Dummy objective
maximize z: n[1,1];

# All numbers in one row have to be different
rows{r in ROWS}:   alldiff{c in COLS} n[r,c];
# All numbers in one column have to be different
cols{c in COLS}:   alldiff{r in ROWS} n[r,c];
# All numbers for each subsquare must be different
squares{sr in 1..BASE, sc in 1..BASE}: alldiff{(c,r) in SUBSQUARES[sr,sc]} n[r,c];

# Fix input data (forces the variable at the corresponding location to have
# the same value as the parameter)
gd{r in ROWS, c in COLS : givenData[r,c] > 0}: n[r,c] = givenData[r,c];

### GUI-related code
The following code creates the Class **SudokuSchama**, a convenient wrapper around ipywidgets to create and display a grid that resembles the game board. It provides functions to get and set the values in the schema itself.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

class SudokuSchema:

    def _createOneGrid(self, startRow : int, startCol : int):
      gridItems = [widgets.VBox([self.items[row, col] for row in range(startRow, startRow+self.BASE)]) for col in range(startCol, startCol+3)]
      return widgets.HBox(gridItems,layout=widgets.Layout(border="solid 2px", width="140px"))
    
    def __init__(self, base):
      self.BASE = base
      self.BSQUARED = base**2

      # Create all widgets
      self.items = {(r,c): widgets.BoundedIntText(
                          value=0,
                          min=0,
                          max=self.BASE**4,
                          step=1,
                          description='',
                          layout = widgets.Layout(width='40px', height='40px')) 
                      for r in range(self.BSQUARED) for c in range(self.BSQUARED)}

      self.sudoku = widgets.HBox([widgets.VBox([self._createOneGrid(r, c) for r in range(0,self.BSQUARED, self.BASE)]) for c in range(0,self.BSQUARED,self.BASE)])

    def display(self):
      display(self.sudoku)

    def getValues(self):
      return {(r+1,c+1) : self.items[r,c].value 
            for r in range(self.BSQUARED) for c in range(self.BSQUARED) if self.items[r,c].value != 0}

    def setValues(self, values : dict):
      for (r,c), v in values.items():
        self.items[r-1, c-1].value = int(v)


### Execute until this cell at first
The following cell creates the sudoku schema - of the size specified in **BASE** and visualizes it. You should then:
1. Fill the cells with the numbers you want then 
2. Execute the following code cell (**solve and display**)

In [ ]:
# Size of a grid (number of items in a row/column = BASE^2)
BASE = 3

# Create and display the grid
sudoku = SudokuSchema(BASE)
sudoku.display()

### Solve and display
After filling the grid with the appropriate numbers, execute the next code cell and.. look up!

In [ ]:
# get a dictionary (row, col) : value  of all the non-zeroes in the grid above
setNumbers = sudoku.getValues()

# If no number has been set don't execute anything, as we probably come from
# an "execute all cells" called at startup
if len(setNumbers) > 0:
    # assign the base (size) and the numbers we got from the grid to the corresponding
    # AMPL entities
    ampl.param["BASE"] = BASE
    ampl.param["givenData"] = setNumbers

    # Solve the model
    ampl.option["solver"]="highs"
    ampl.solve()

    # Get the data from AMPL and assign them to the entities making up the 
    # grid above
    values = ampl.get_data("n").to_dict()
    sudoku.setValues(values)